In [ ]:
# from geopy.geocoders import Nominatim
from geopy import *
import geopy.distance
import pandas as pd
import numpy as np
import math
import itertools
import re
import Levenshtein

In [ ]:
excel = r'2020/transmission.xlsx'
sheet = 'Trans.Investments'

df = pd.read_excel(excel, sheet_name=sheet, header=[1])

# TODO: also contains new substations!

wanted_columns = ['Investment number',
                  'This investment belongs to project number…',
                  'Commissioning Year',
                  'Status ID\n1 : Under Consideration,\n2 : In Planning but not permitting,\n3 : In permitting,\n4 : Under Construction',
                  'Type of Element',
                  'Substation From',
                  'Substation To',
                  'Technology',
                  'Total route length (km)']
status_column = wanted_columns[3]
df[wanted_columns]

In [ ]:
wanted    = df[wanted_columns]
# only choose those in permitting or under construction
wanted    = wanted.loc[wanted[status_column].astype(int) >= 3]

ac_lines  = wanted.query("`Type of Element` == 'ACTransmissionLine'")
dc_lines  = wanted.query("`Type of Element` == 'DCTransmissionLine'")
on_subst  = wanted.query("`Type of Element` == 'OnshoreSubstation'")
off_subst = wanted.query("`Type of Element` == 'OffshoreSubstation'")

In [ ]:
# For the time being, remove lines connected to new substations
new_subst = set(on_subst['Substation From']).union(on_subst['Substation To'])
ac_lines  = ac_lines.query("`Substation From` not in @new_subst")
ac_lines  = ac_lines.query("`Substation To` not in @new_subst")

# Use bus names from buses.csv (v0.1.0)
See https://github.com/PyPSA/pypsa-eur/blob/v0.1.0rc/data/entsoegridkit/buses.csv. Data is from 2017 (newer gridkit extracts do not contain 'tags' with substation names).

In [ ]:
buses_file = 'buses_v0.1.0.csv'

# see base_network.py in PyPSA-Eur repository
buses = (pd.read_csv(buses_file, quotechar="'",
                     true_values='t', false_values='f',
                     dtype=dict(bus_id="str"))
        .set_index("bus_id")
        .drop(['station_id'], axis=1)
        .rename(columns=dict(voltage='v_nom')))

In [ ]:
no_tags = buses.tags.isnull().sum()
print(f'{no_tags} buses have no tags.')

yes_tags = buses.tags.notnull().sum()
print(f'{yes_tags} buses have tags.')

In [ ]:
buses = buses.query('tags.notnull()', engine='python')
buses = buses.query("symbol == 'Substation'")

# Extract 'name_eng' and 'country' from tags in  buses

In [ ]:
split_regex = r'("\w+"=>"[^"]*"),' # Form: 'key => value, key => value, ...'

tag_regex   = r'"(?P<key>\w+)"=>"(?P<value>[^"]*)"' # Form: 'key => value'
tag_pattern = re.compile(tag_regex)

rows = []

for index, row in buses.iterrows():
    name    = ''
    country = ''
    x = row['x']
    y = row['y']
    
    tags_string = row['tags']
    
    tags = re.split(split_regex, tags_string)
    
    # Remove whitespaces at front and end, remove None values
    tags = [s.strip() for s in tags]
    tags = list(filter(None, tags))
    
    for tag in tags:
        m = tag_pattern.match(tag)
            
        if m is None:
            print(tag)
            
        # see group names in tag_regex
        key   = m.group('key')
        value = m.group('value')
        
        if key == 'name_eng':
            name = value.strip()
        elif key == 'country':
            country = value.strip()
    
    if name == 'unknown' or not name:
        continue
        
    rows.append((name, country, x, y))

In [ ]:
curated_buses = pd.DataFrame.from_records(rows, columns=['name', 'country', 'x', 'y'])

## Remove duplicate rows

In [ ]:
curated_buses = curated_buses.loc[~curated_buses.duplicated()]

## There are substations which share the same name but have different coordinates
- large deviation between coordinates => substations are most likely in different countries 
    - BUT: it does occur that different places in the same country get the same name
- small deviation between coordinates => reference to same substation (error in gridextract?)

In [ ]:
pypsa_countries = ['AL', 'AT', 'BA', 'BE', 'BG', 'CH', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GB', 'GR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'ME', 'MK', 'NL', 'NO', 'PL', 'PT', 'RO', 'RS', 'SE', 'SI', 'SK']

### List of all duplicates

In [ ]:
duplicated = curated_buses.loc[curated_buses.name.duplicated()]
duplicated = duplicated.query("country in @pypsa_countries")

for name in duplicated.name.unique():
    print(name)
    for index, row in curated_buses.query('name == @name').iterrows():
        print(f"({row['x']}, {row['y']}), {row['country']}")
    print('----')

### Same name and country, large deviations

In [ ]:
curated_buses.query("name == 'Yuzhnaya'")

### Same name, different country, large deviation

In [ ]:
curated_buses.query("name == 'Saida'")

In [ ]:
curated_buses.query("name == 'Titan'")

## (TODO) Add new substations

In [ ]:
on_subst

# extract country if it matches regex
# otherwise, np.NAN

## Remove '(\<Country Code\>) ' from tyndp substation name strings, add new column instead
Otherwise, this could negatively impact the Levenshtein distance.

In [ ]:
subst_regex   = r'(?P<place>.+)\s?[\[(](?P<country>\w{2})[)\]]' # Form: 'Glorenza (IT)'
subst_pattern = re.compile(subst_regex)

# use this if other pattern does not match to remove comments in parentheses
# e.g. 'Molai (through Sklavouna Terminal)'
alt_regex   = r'(?P<place>.+)\s?[\[(].*[)\]]'
alt_pattern = re.compile(alt_regex)

fr_names     = []
fr_countries = []
to_names     = []
to_countries = []

for index, row in ac_lines.iterrows():    
    fr = row['Substation From']
    to = row['Substation To']
    
    # default values if regex does not match
    fr_name = fr
    to_name = to    
    fr_country = np.NAN
    to_country = np.NAN
    
    fr_match = subst_pattern.match(fr)
    to_match = subst_pattern.match(to)
    
    if fr_match:
        fr_name    = fr_match.group('place').strip()
        fr_country = fr_match.group('country').strip()
    else:
        fr_alt_match = alt_pattern.match(fr)
        if fr_alt_match:
            fr_name = fr_alt_match.group('place')
        
    if to_match:
        to_name    = to_match.group('place').strip()
        to_country = to_match.group('country').strip()
    else:        
        to_alt_match = alt_pattern.match(to)
        if to_alt_match:
            to_name = to_alt_match.group('place')
        
    fr_names.append(fr_name)
    fr_countries.append(fr_country)
    to_names.append(to_name)
    to_countries.append(to_country)

In [ ]:
ac_lines['Substation From'] = fr_names
ac_lines['Substation To'] = to_names
ac_lines['Country1'] = fr_countries
ac_lines['Country2'] = to_countries
ac_lines

## create mapping from all unique tyndp substation names to substation names from 'buses'

In [ ]:
tyndp_subs   = set(ac_lines['Substation From']).union(set(ac_lines['Substation To']))
tyndp_to_bus = {}

for tyndp in tyndp_subs:
    buses_subs = curated_buses.name.values
    
    closest = min([(bus, Levenshtein.distance(bus.lower(), tyndp.lower())) for bus in buses_subs], key=lambda t: t[1])[0]
    
    tyndp_to_bus[tyndp] = closest

In [ ]:
tyndp_to_bus

### Helper functions: Out of all possible pairs of locations from two lists, get the pair whose distance is closest to the specified (line) length
Deals with problem of multiple places in same country sharing a name.

In [ ]:
def extract_coords(rows):
    coordinates = []
    for index, row in rows.iterrows():
        coordinates.append((row['x'], row['y']))
    return coordinates

In [ ]:
def match_pair_with_length(s1_rows, s2_rows, length):
    s1_coords = extract_coords(s1_rows)
    s2_coords = extract_coords(s2_rows)
    
    combinations  = list(itertools.product(s1_coords, s2_coords))
    with_distance = [(a, b, geopy.distance.distance(a,b).km) for (a,b) in combinations]
    
    best_match = min(with_distance, key=lambda t: abs(length - t[2]))
    return best_match

In [ ]:
# Example
buses_s1 = curated_buses.loc[curated_buses['name'] == 'Yuzhnaya']
buses_s2 = curated_buses.loc[curated_buses['name'] == 'Liteynaya']

best = match_pair_with_length(buses_s1, buses_s2, 430)
best

# Match start- and endpoints of lines to substations from buses.csv

In [ ]:
fr_to_tuples = {}
error_tuples = {}

for index, row in ac_lines.iterrows():    
    # Get closest name match based on Levenshtein distance for start- and endpoints of line
    fr = row['Substation From']
    to = row['Substation To']
    
    fr_country = row['Country1']
    to_country = row['Country2']
            
    s1 = tyndp_to_bus[fr]
    s2 = tyndp_to_bus[to]
    
    # Extract respective rows in buses to determine coordinates
    buses_s1 = curated_buses.loc[curated_buses.name == s1]
    buses_s2 = curated_buses.loc[curated_buses.name == s2]
    
    # If we were able to extract country from name, restrict chosen rows to this country.
    if not pd.isna(fr_country):
        buses_s1 = buses_s1.loc[buses_s1['country'] == fr_country]
    if not pd.isna(to_country):
        buses_s2 = buses_s2.loc[buses_s2['country'] == to_country]
    
    # TODO: use at least one!  
    if buses_s1.empty or buses_s2.empty:
        # print(f'{fr}, {fr_country} <-> {to}, {to_country}')
        tpl = (s1, np.NAN, np.NAN, s2, np.NAN, np.NAN, np.NAN)
        error_tuples[index] = tpl
        continue
    
    # Choose pair which matches length best
    length = row['Total route length (km)']
    (x1, y1), (x2, y2), coord_dist = match_pair_with_length(buses_s1, buses_s2, length)
        
    tpl = (s1, x1, y1, s2, x2, y2, coord_dist)
    
    # TODO: how to choose an appropriate tolerance?
    if not math.isclose(coord_dist, length, rel_tol=0.45):
        error_tuples[index] = tpl
    else:
        fr_to_tuples[index] = tpl

In [ ]:
coordinates = pd.DataFrame(index=fr_to_tuples.keys(), data=fr_to_tuples.values(), columns=['s1', 'x1', 'y1', 's2', 'x2', 'y2', 'coord_dist'])

result = ac_lines.copy()
result = result.join(coordinates)

percentage = coordinates.index.size / ac_lines.index.size
print(f'{percentage * 100}% of lines are probably correct.')

print('Lines where we probably found the correct coordinates:')
result.loc[~result.s1.isna()]

In [ ]:
error_lines = result.loc[result.s1.isna()]
error_subst = set(error_lines['Substation From']).union(error_lines['Substation To'])

print('')
{(k,tyndp_to_bus[k]) for k in error_subst}

# Determine coordinates using geopy

In [ ]:
locator = Nominatim(user_agent='esm_group')
geocode = RateLimiter(locator.geocode, min_delay_seconds=0.05)

In [ ]:
x1 = []
y1 = []
x2 = []
y2 = []

error_rows = []

for index, row in ac_lines.iterrows():
    fr   = row['Substation From']
    to   = row['Substation To']
    dist = row['Total route length (km)']

    fr_loc = geocode(fr)
    to_loc = geocode(to)

    if fr_loc is None or to_loc is None:
        error_rows.append([row.values])
        continue
        
    fr_coords  = fr_loc.latitude, fr_loc.longitude
    to_coords  = to_loc.latitude, to_loc.longitude
    coord_dist = distance.distance(fr_coords, to_coords).km

    if math.isclose(coord_dist, dist, rel_tol=0.25):
        x1.append(fr_coords[0])
        y1.append(fr_coords[1])
        
        x2.append(to_coords[0])
        y2.append(to_coords[1])
    else:
        error_rows.append([row.values])
        # x1.append(np.NAN)
        # y1.append(np.NAN)
        
        # x2.append(np.NAN)
        # y2.append(np.NAN)

In [ ]:
len(x1), len(error_rows)

In [ ]:
error_rows